In [2]:
import ROOT as r
import math as m
import os

Welcome to JupyROOT 6.18/00


In [3]:
Output = r.TChain("output")

directory = "/global/projecta/projectdirs/atlas/eressegu/Ntuple_h026_0630/data15/data/data15/root"
for filename in os.listdir(directory):
    f = os.path.join(directory, filename)
    if os.path.isfile(f):
        Output.Add(f)

In [4]:
photon1 = r.TLorentzVector()
photon2 = r.TLorentzVector()
diphoton = r.TLorentzVector()

h_thetaphi_sb1 = r.TH2F("h_thetaphi_sb1", "thetavsphi", 60,-180, 180, 60, -90, 90)
h_thetaphi_sb2 = r.TH2F("h_thetaphi_sb2", "thetavsphi", 60,-180, 180, 60, -90, 90)
h_thetaphi_sw = r.TH2F("h_thetaphi_sw", "thetavsphi", 60,-180, 180, 60, -90, 90)
hist3 = r.TH1F("hist3", "higgs mass", 60, 0, 600)
hist4 = r.TH2F("hist4", "photon emissions", 60,-180, 180, 60, -90, 90)
hist5 = r.TH1F("hist5", "higgs phi", 60, -180, 180)
hist6 = r.TH1F("hist6", "higgs rapidity" , 60, -3, 3)

In [6]:
for entry in Output:
    if entry.isPassed:
        photon1.SetPtEtaPhiE(entry.ph_pt1, entry.ph_eta1, entry.ph_phi1, entry.ph_E1)
        photon2.SetPtEtaPhiE(entry.ph_pt2, entry.ph_eta2, entry.ph_phi2, entry.ph_E2)
        diphoton = photon1 + photon2
        angle = diphoton.Pt() / diphoton.Pz()
        actual_angle = m.atan(angle)
        phi = diphoton.Phi() * 180 / m.pi
        rapidity = diphoton.Rapidity()
        hist3.Fill(diphoton.M())
        hist5.Fill(phi)
        hist6.Fill(rapidity)
        hist4.Fill(phi, actual_angle * 180 / m.pi)
        lowerb = 100.0
        lowert = 120.0
        upperb = 130.0
        uppert = 150.0
        if diphoton.M() >= lowerb and diphoton.M() <= lowert:
            h_thetaphi_sb1.Fill(phi, actual_angle * 180 / m.pi)
        elif diphoton.M() >= upperb and diphoton.M() <= uppert:
            h_thetaphi_sb2.Fill(phi, actual_angle * 180 / m.pi)
        elif diphoton.M() >= lowert and diphoton.M() <= upperb:
            h_thetaphi_sw.Fill(phi, actual_angle * 180 / m.pi)
        

canvas_4 = r.TCanvas("tvpw", "tvpw")
hist4.Draw("colz")
hist4.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
hist4.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_4.SaveAs("ThetavPhiRaw.png")

canvas_5 = r.TCanvas("higgs_phi", "higgs_phi")
hist5.SetLineWidth(2)
hist5.SetLineColor(2)
hist5.SetLineStyle(1)
hist5.SetFillColor(2)
hist5.Draw()
hist5.GetXaxis().SetTitle("Phi_{#gamma#gamma} [GeV]")
hist5.GetYaxis().SetTitle("Entries")
canvas_5.SaveAs("Higgs_Phi.png")

canvas_6 = r.TCanvas("higgs_rapidity", "higgs_rapidity")
hist6.SetLineWidth(2)
hist6.SetLineColor(2)
hist6.SetLineStyle(1)
hist6.SetFillColor(2)
hist6.Draw()
hist6.GetXaxis().SetTitle("Rapidity_{#gamma#gamma}")
hist6.GetYaxis().SetTitle("Entries")
canvas_6.SaveAs("Higgs_Rapidity.png")


Warning in <TCanvas::Constructor>: Deleting canvas with same name: tvpw
Info in <TCanvas::Print>: png file ThetavPhiRaw.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: higgs_phi
Info in <TCanvas::Print>: png file Higgs_Phi.png has been created
Warning in <TCanvas::Constructor>: Deleting canvas with same name: higgs_rapidity
Info in <TCanvas::Print>: png file Higgs_Rapidity.png has been created


In [7]:
canvas_3 = r.TCanvas("higgs_mass", "higgs_mass")
hist3.SetLineWidth(2)
hist3.SetLineColor(r.kBlack)
hist3.Draw("pe")
hist3.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
hist3.GetYaxis().SetTitle("Entries")
canvas_3.SaveAs("higgs_mass.png")



Info in <TCanvas::Print>: png file higgs_mass.png has been created


In [8]:
fit = r.TF1("expofit", "expo", 100, 150)
hist3.Fit(fit, "S", "", 100, 120)

 FCN=1.14442e-27 FROM MIGRAD    STATUS=CONVERGED      28 CALLS          29 TOTAL
                     EDM=2.16774e-22    STRATEGY= 1      ERROR MATRIX ACCURATE 
  EXT PARAMETER                                   STEP         FIRST   
  NO.   NAME      VALUE            ERROR          SIZE      DERIVATIVE 
   1  Constant     4.94274e+00   1.86497e-01   0.00000e+00   4.76757e-09
   2  Slope        4.11574e-02   1.67830e-03  -0.00000e+00   5.31263e-07


In [9]:
N1 = fit.Integral(100, 120)
print(N1)
N3 = fit.Integral(130, 150)
print(N3)
N2 = fit.Integral(120, 130)
print(N2)

266693.07065148244
916735.8247939995
242084.1494377998


In [11]:
print(hist3.Integral(11, 12))
print(hist3.Integral(14, 15))
print(hist3.Integral(13, 13))
print(hist3.Integral(11, 15))

26482.0
14642.0
11766.0
52890.0


In [12]:
print(hist3.Integral(13, 13)/hist3.Integral(11, 12))
print(hist3.Integral(13, 13)/hist3.Integral(14, 15))

0.44430178989502306
0.8035787460729409


In [13]:
w1 = N2/N1
w2 = N2/N3
print(w1, w2)

0.9077256819850341 0.26407187642328556


In [14]:
hist31 = r.TH1F("hist", "higgs mass", 60, 0, 600)
#for entry in Output:
    #photon1.SetPtEtaPhiE(entry.ph_pt1, entry.ph_eta1, entry.ph_phi1, entry.ph_E1)
    #photon2.SetPtEtaPhiE(entry.ph_pt2, entry.ph_eta2, entry.ph_phi2, entry.ph_E2)
    #diphoton = photon1 + photon2
    #hist31.Fill(diphoton.M(), w1 * .9)
canvas_31 = r.TCanvas("higgs_mass_weight", "higgs_mass_weight")
hist31.SetLineWidth(2)
hist31.SetLineColor(2)
hist31.SetLineStyle(1)
hist31.SetFillColor(2)
hist31.Draw()
hist31.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
hist31.GetYaxis().SetTitle("Entries")
canvas_31.SaveAs("Higgs_mass_weight.png")


Info in <TCanvas::Print>: png file Higgs_mass_weight.png has been created


In [15]:
mass_clone = hist3.Clone("mass_clone")
mass_clone.Add(hist31, -1)
canvas_32 = r.TCanvas("mass_subtracted", "mass_subtracted")
mass_clone.SetLineWidth(2)
mass_clone.SetLineColor(2)
mass_clone.SetLineStyle(1)
mass_clone.SetFillColor(2)
mass_clone.Draw()
mass_clone.GetXaxis().SetTitle("m_{#gamma#gamma} [GeV]")
mass_clone.GetYaxis().SetTitle("Entries")
canvas_32.SaveAs("mass_subtracted.png")

Info in <TCanvas::Print>: png file mass_subtracted.png has been created


In [16]:
print(hist3.Integral(11, 12))
print(hist3.Integral(14, 15))
print(hist3.Integral(13, 13))
print(hist3.Integral(11, 15))
print(h_thetaphi_sb1.Integral())
print(h_thetaphi_sb2.Integral())
print(h_thetaphi_sw.Integral())

26482.0
14642.0
11766.0
52890.0
26482.0
14642.0
11766.0


In [17]:
h_thetaphi_sb1.Scale(w1/2)
h_thetaphi_sb2.Scale(w2/2)
h_thetaphi_sb = h_thetaphi_sb1.Clone("h_thetaphi_sb")
h_thetaphi_sb.Add(h_thetaphi_sb2)
h_thetaphi_s = h_thetaphi_sw.Clone("h_thetaphi_s")
h_thetaphi_s.Add(h_thetaphi_sb, -1)
canvas_thetaphi = r.TCanvas("tvp_subtracted", "tvp_subtracted")
h_thetaphi_s.Draw("colz")
h_thetaphi_s.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_thetaphi_s.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("thetavphisubtracted.png")

h_thetaphi_sb.Draw("colz")
h_thetaphi_sb.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_thetaphi_sb.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("thetavphi_setbands.png")

h_thetaphi_sb1.Draw("colz")
h_thetaphi_sb1.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_thetaphi_sb1.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("thetavphi_setband1.png")

h_thetaphi_sb2.Draw("colz")
h_thetaphi_sb2.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_thetaphi_sb2.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("thetavphi_setband2.png")

h_thetaphi_sw.Draw("colz")
h_thetaphi_sw.GetXaxis().SetTitle("#phi_{#gamma#gamma}")
h_thetaphi_sw.GetYaxis().SetTitle("#theta_{#gamma#gamma}")
canvas_thetaphi.SaveAs("thetavphi_signal_window.png")
 

Info in <TCanvas::Print>: png file thetavphisubtracted.png has been created
Info in <TCanvas::Print>: png file thetavphi_setbands.png has been created
Info in <TCanvas::Print>: png file thetavphi_setband1.png has been created
Info in <TCanvas::Print>: png file thetavphi_setband2.png has been created
Info in <TCanvas::Print>: png file thetavphi_signal_window.png has been created


In [18]:
print(h_thetaphi_s.Integral())
print(h_thetaphi_sw.Integral())
print(h_thetaphi_sb1.Integral())
print(h_thetaphi_sb2.Integral())
print(h_thetaphi_sb.Integral())

-2186.46599984169
11766.0
12019.195722460747
1933.270228087902
13952.465986847878
